Changelog

1. Created a basic 'room' class
2. Created a basic 'object' class and a basic 'player' class
3. Added look, showMessages, move, getInstruction, inputSplit functions to Player. Player can now check for 
   and respond to verbs. Added dummy verb LLAMA to test, and verb E to allow movement East. Which works!
   Changed directions in Room to be a dictionary so as to be able to move to arbitrary rooms rather than 
   performing calculations as in the original. A little more work to set up (and I've commented a load out
   for now!) but far easier to actually use. A few tweaks for the main loop to show messages and 'look' at 
   each round.
4. Added directional numbers for whole map. Removed debug info from E. Tweaked messages. Added DEBUG flag and
   implemented clear_output().
5. Added other direction verbs. Tweaked direction method to use single logical subroutine. Remembered that you
   can't append using commas in functions other than print...
6. Added the get verb.
7. Added the help verb. This is quite different to the original implementation, being written rather than 
   generated. Removed the LLAMA test verb.
8. Added the carrying verb. Tweaked the input splitting to remove (eg) ?'s from input verbs to match original.
9. Noted that GO (which will be ignored for now) and other movement verbs have various flag checks that are not
   yet implemented. Will do these as and when implementing the verbs associated with them (hopefully!). 
   Also, implemented TAKE as a pseduonym for GET.
10. Implemented OPEN. Open needs a phrase (something to open). Tweaked Room to not show invisible items. Tweaked input splitter to include spaces in the phrase. Coded in logic for openable things (coffin, desk/drawer, door) and made things (candle/ring) 'appear' when the container is opened. Tweaked logic of GET to handle when things are taken. Started moving object IDs over to constants to make things a wee bit more proper.
11. Implemented EXAMINE verb. Added 'makeVisible' with a parameter of object ID to make things visible when they are found, seems I'm doing this a bit! Also added a changeDesc(ription) function to change object descs. Implemented these in the OPEN verb. Implemented JUMP secret verb to jump to another room to help test. Went back and changed OPEN to make use of the new subroutines.
12. Implemented READ verb. Added an isHere() routine to produce the checking for presense code at the start of each verb. Tweaked catch-all for OPEN. Added the catch-all for EXAMINE. Added isCarrying test and setFlag. Used isHere in previous verbs.
13. Implemented SAY verb.
14. Implemented DIG verb. Added changeDesc() and addExit() for rooms.
15. Implemented SWING verb.
16. Implemented CLIMB verb. This is as-per original, so the rope has to just sit there (no way of dropping it) and  at the moment you can move away regardless of whether up or down the tree. Changed changelog to markdown.
17. Implemented LIGHT verb. Added catch-all to CLIMB. Added the CANDLELIT flag. Tweaked CARRYING to only show the header once.
18. Implemented UNLIGHT verb.
19. Implemented SPRAY verb. Tweaked isHere to accept phrase or ID. Updated the help.
20. Implemented USE verb. Added isVisible check. Tweaked spray to use visibility check rather than flag. Added makeInvisible.
21. Implemented UNLOCK verb. Tweaked OPEN to allow the door to open.
   
Future Thoughts
1. I think I'll implement onEnter and onLeave events at some point, called by the move() function of player.
2. Do something with the rope/tree to make it more useable.
3. Sort out the light issue - make it useful!
4. Need to make the bats appear and work.
5. Need to make the ghosts appear and work.

Room class

In [ ]:
class Room:
    def __init__(self,id,desc,exits,contains):
        self.id = id
        self.desc = desc
        self.exits = exits
        self.contains = contains
        
    def describe(self):
        print ("HAUNTED HOUSE")
        print ("-------------")
        print ("YOUR LOCATION:",self.desc)
        print ("EXITS:",", ".join(self.exits))
        contains = []
        for c in self.contains:
            if c.visible:
                contains.append(c.desc)
        if (contains == []):
            contains.append('NOTHING')
        print ("CONTAINS:",", ".join(contains))

    def changeDesc(self,newDesc):
        self.desc = newDesc
        
    def addExit(self,direction,target):
        self.exits[direction] = target

Object class

In [ ]:
class Object:
    def __init__(self,id,desc,gettable=False,visible=True):
        self.id = id
        self.desc = desc
        self.gettable = gettable
        self.visible = visible
        
    def moveTo(self,moveToObject):
        moveToObject.contains.append(self)

Player class 

In [ ]:
class Player:
    def __init__(self,room,carrying = [],won=False):
        self.room = room
        self.carrying = carrying
        self.won = False
        self.messages = []
        self.flags = {}
        
    def look(self):
        rooms[self.room].describe()
        
    def showMessages(self):
        if (len(self.messages) != 0):
            for message in self.messages:
                print(message)
            print()
        self.messages = []
        
    def move(self,room):
        self.room = room
        
    def getInstruction(self):
        self.instruction = input("WHAT WILL YOU DO NOW? ")
        phraseLen = self.inputSplit()
        if (phraseLen == 0):
            self.messages.append("IT HELPS IF YOU WRITE SOMETHING")
        else:
            #They've written something, try and do it
            try:
                #print('trying: '+self.verb)
                eval('self.'+self.verb+'()')
            except:
                print('SENSE ISSUE')
                self.messages.append("YOU DON'T MAKE SENSE") #verb doesn't exist
        
    def inputSplit(self):
        #print('You wrote:',self.instruction)
        words = self.instruction.upper().split(" ")
        self.instruction = ''
        if (len(words) == 0):
            self.verb = ''
            self.phrase = ''
            return 0
        elif (len(words) == 1):
            self.verb = words[0]
            self.verb = re.sub(r'\W+', '', self.verb)
            self.phrase = ''
            return 1
        else:
            self.verb = words[0]
            self.verb = re.sub(r'\W+', '', self.verb)
            self.phrase = " ".join(words[1:])
            return 2
        
    def goDirection(self,letter,word):
        if (letter in rooms[self.room].exits):
            self.move(rooms[self.room].exits[letter])
            self.messages.append('YOU GO '+word)
        else:
            self.messages.append('YOU CANNOT GO '+word+' FROM HERE')

    def makeVisible(self,invisibleObjectKey):
        for i in range(len(rooms[self.room].contains)):
            ob = rooms[self.room].contains[i]
            if ob.id == invisibleObjectKey:
                ob.visible = True
                
    def makeInvisible(self,visibleObjectKey):
        for i in range(len(rooms[self.room].contains)):
            ob = rooms[self.room].contains[i]
            if ob.id == visibleObjectKey:
                ob.visible = False
                
    def changeDesc(self,objectKey,newDesc):
        for i in range(len(rooms[self.room].contains)):
            ob = rooms[self.room].contains[i]
            if ob.id == objectKey:
                ob.desc = newDesc
                
    def isHere(self,objectPhrase):
        for i in range(len(rooms[self.room].contains)):
            ob = rooms[self.room].contains[i]
            if ob.desc == objectPhrase or ob.id == objectPhrase:
                return True
        return False
    
    def isVisible(self,objectPhrase):
        for i in range(len(rooms[self.room].contains)):
            ob = rooms[self.room].contains[i]
            if ob.desc == objectPhrase or ob.id == objectPhrase:
                return ob.visible
    
    def isCarrying(self,objectKey):
        for i in range(len(self.carrying)):
            ob = self.carrying[i]
            if ob.id == objectKey or ob.desc == objectKey:
                return True
        return False

    def setFlag(self,flag,value):
        self.flags[flag] = value
        
    '''VERBS'''
    def JUMP(self):
        #secret verb to help testing - can JUMP X to jump to room X
        if DEBUG == 1:
            self.room = int(self.phrase)
        else:
            self.messages.append('YOU CAN ONLY JUMP IN DEBUG MODE')
    
    def E(self):
        self.goDirection('E','EAST')
    def W(self):
        self.goDirection('W','WEST')
    def N(self):
        self.goDirection('N','NORTH')
    def S(self):
        self.goDirection('S','SOUTH')
    def U(self):
        self.goDirection('U','UP')
    def D(self):
        self.goDirection('D','DOWN')
            
    def LOOK(self):
        if (self.phrase == ''):
            self.messages.append('YOU LOOK AROUND')
            
    def GET(self):
        found = False
        numObjects = len(rooms[self.room].contains)
        for i in range(numObjects):
            ob = rooms[self.room].contains[i]
            if ob.desc == self.phrase:
                found = True
                if ob.gettable:
                    self.carrying.append(ob)
                    del rooms[self.room].contains[i]
                    self.messages.append('YOU GET '+ob.desc)
                else:
                    self.messages.append('YOU CANNOT GET '+ob.desc)
                break
        if not found:
            self.messages.append('COULD NOT FIND '+self.phrase)
    
    def TAKE(self):
        self.GET()
        
    def HELP(self):
        self.messages.append("HELP")
        self.messages.append("====")
        self.messages.append("Can you get out of the haunted house?")
        self.messages.append("You can move in (up to) six directions using the initial letter of the direction.")
        self.messages.append("(N)orth, (S)outh, (E)ast, (W)est, (U)p, (D)own.")
        self.messages.append("Commands:")
        self.messages.append("HELP lists this help file.")
        self.messages.append("LOOK looks around the room you are in.")
        self.messages.append("GET/TAKE OBJECT Gets an object you can see if possible and puts in your inventory.")
        self.messages.append('CARRYING? Tells you the contents of your inventory.')
        self.messages.append('OPEN OBJECT opens an object, door, etc.')
        self.messages.append('EXAMINE OBJECT takes a good look at an object.')
        self.messages.append('READ OBJECT reads any text on an object.')
        self.messages.append('SAY PHRASE says a phrase. Should you need to.')
        self.messages.append('DIG tries to dig a hole.')
        self.messages.append('SWING / SWING OBJECT lets you swing yourself or something.')
        self.messages.append('CLIMB OBJECT lets you try to climb something.')
        self.messages.append('LIGHT OBJECT lets you light something flammable.')
        self.messages.append('UNLIGHT puts out a flame.')
        self.messages.append('SPRAY OBJECT tries to spray something with a sprayable.')
        self.messages.append('USE OBJECT tries to use an object - if it is in a usable state.')
        self.messages.append('UNLOCK OBJECT tries to unlock something which is locked.')
        
    def CARRYING(self):
        numCarrying = len(self.carrying)
        if numCarrying == 0:
            self.messages.append("YOU ARE NOT CARRYING ANYTHING")
        else:
            self.messages.append("YOUR INVENTORY CONTAINS:")
            for i in range(numCarrying):
                self.messages.append(" - "+self.carrying[i].desc)
                
    def OPEN(self):
        if self.phrase == '':
            self.messages.append('OPEN WHAT?')
        else:
            if not self.isHere(self.phrase):
                self.messages.append('MAYBE YOU SHOULD GO TO WHERE THERE IS A '+self.phrase+' BEFORE TRYING TO OPEN ONE?')
            else:
                self.messages.append('YOU TRY TO OPEN THE '+self.phrase)
                if self.phrase == 'DRAWER' or self.phrase == 'DESK':
                    self.messages.append('YOU OPEN THE DESK AND SEE A CANDLE INSIDE.')
                    self.changeDesc(DESK,'DESK WITH OPEN DRAWER')
                    self.makeVisible(CANDLE)
                elif self.phrase == 'DOOR':
                    if self.flags['DOORLOCKED']:
                        self.messages.append('IT IS LOCKED')
                    else:
                        self.messages.append('YOU OPEN THE DOOR')
                elif self.phrase == 'COFFIN':
                    self.messages.append('YOU OPEN THE COFFIN LID AND SEE A RING INSIDE.')
                    self.changeDesc(COFFIN,'OPEN COFFIN')
                    self.makeVisible(RING)
                    self.messages.append('THAT IS CREEPY!')
                else:
                    self.messages.append('YOU CANNOT OPEN THE '+self.phrase)
                        
    def EXAMINE(self):
        if self.phrase == '':
            self.messages.append('EXAMINE WHAT?')
        else:
            if not self.isHere(self.phrase):
                self.messages.append('MAYBE YOU SHOULD EXAMINE SOMETHING THAT IS HERE?')
            else:
                self.messages.append('YOU TRY TO EXAMINE THE '+self.phrase)
                if self.phrase == 'COAT':
                    self.messages.append('YOU EXAMINE THE COAT AND FIND A KEY IN THE POCKET')
                    self.makeVisible(KEY)
                elif self.phrase == 'RUBBISH':
                    self.messages.append('YOU EXAMINE THE PILE OF RUBBISH')
                    self.messages.append('THAT IS DISGUSTING!')
                elif self.phrase == 'DESK' or self.phrase == 'DRAWER':
                    self.messages.append('THERE IS A DRAWER')
                elif self.phrase == 'BOOKS' or self.phrase == 'SCROLL':
                    self.messages.append('YOU EXAMINE THE '+self.phrase)
                    self.messages.append('SOME WORDS. MAYBE YOU COULD READ THEM?')
                elif self.phrase == 'WALL':
                    self.messages.append('YOU EXAMINE THE WALL')
                    self.messages.append('THERE IS SOMETHING BEYOND')
                elif self.phrase == 'COFFIN':
                    self.messages.append('YOU EXAMINE THE COFFIN')
                    self.messages.append('THE LID LOOKS A BIT LOOSE')
                else:
                    self.messages.append('YOU CANNOT EXAMINE THE '+self.phrase)
                    
    def READ(self):
        if self.phrase == '':
            self.messages.append('READ WHAT?')
        else:
            if not self.isHere(self.phrase) and not self.isCarrying(self.phrase):
                self.messages.append('PERHAPS YOU SHOULD READ SOMETHING THAT IS ACTUALLY HERE?')
            else:
                self.messages.append('YOU TRY TO READ THE '+self.phrase)
                if self.phrase == 'BOOKS':
                    self.messages.append('THEY ARE DEMONIC WORKS!')
                elif self.phrase == 'MAGIC SPELLS' and self.flags['SAIDMAGICIN45'] != True and self.isCarrying(MAGICSPELLS):
                    self.messages.append('USE THIS WORD WITH CARE "'+MAGICWORD+'"')
                elif self.phrase == 'SCROLL' and self.isCarrying(SCROLL):
                    self.messages.append('THE SCRIPT IS IN AN ALIEN TONGUE')
                else:
                    self.messages.append('YOU CANNOT READ THE '+self.phrase)
                    
    def SAY(self):
        if self.phrase == '':
            self.messages.append('SAY WHAT?')
        else:
            if self.isCarrying(MAGICSPELLS) and self.phrase == MAGICWORD:
                self.messages.append('*** MAGIC OCCURS ***')
                if self.room != 45:
                    self.messages.append('THIS ROOM DOES NOT FEEL LIKE SOMEWHERE SAFE TO BE DOING MAGIC!')
                    self.move(randint(0,63))
                else:
                    player.setFlag('SAIDMAGICIN45',True)
            else:
                self.messages.append('OK. '+self.phrase)
                
    def DIG(self):
        if self.isCarrying(SHOVEL):
            if self.room == 30:
                rooms[self.room].changeDesc('CELLAR WITH BARRED WINDOW AND A HOLE IN THE WALL')
                rooms[self.room].addExit('E',31)
            else:
                self.messages.append('YOU DIG A HOLE')
                self.messages.append('\nTHAT... LOOKS A BIT DANGEROUS...')
                self.messages.append('\nYOU FILL THE HOLE IN AGAIN')
        else:
            self.messages.append('YOU HAVE NOTHING TO DIG WITH')
            
    def SWING(self):
        if self.phrase == 'ROPE' and self.room == 7 and not self.isCarrying(ROPE):
            self.messages.append('THIS IS NO TIME TO BE PLAYING GAMES')
        elif self.phrase == 'ROPE' and self.isCarrying(ROPE):
            self.messages.append('YOU SWUNG IT')
        elif self.phrase == 'AXE' and self.isCarrying(AXE) and self.room == 43:
            rooms[self.room].addExit('N',35)
            rooms[self.room].changeDesc('STUDY WITH DESK & SECRET ROOM')
            self.messages.append('YOU BROKE DOWN THE WALL')
        elif self.phrase == 'AXE' and self.isCarrying(AXE):
            self.messages.append('WHOOSH!')
        else:
            self.messages.append('YOU CANNOT SWING HERE. SORRY.')

    def CLIMB(self):
        if self.phrase == '':
            self.messages.append('CLIMB WHAT?')
        else:
            if self.phrase == 'ROPE' and self.isCarrying(ROPE):
                self.messages.append('IT ISN\'T ATTACHED TO ANYTHING')
            elif self.phrase == 'ROPE' and not self.isCarrying(ROPE) and self.room == 7 and self.flags['ISUPTREE'] == False:
                self.messages.append('YOU SEE THICK FOREST AND CLIFF SOUTH')
                self.setFlag('ISUPTREE',True)
            elif self.phrase == 'ROPE' and not self.isCarrying(ROPE) and self.room == 7 and self.flags['ISUPTREE'] == True:
                self.messages.append('GOING DOWN!')
                self.setFlag('ISUPTREE',False)
            else:
                self.messages.appned('YOU CANNOT CLIMB THE '+self.phrase)
                
    def LIGHT(self):
        if self.phrase == '':
            self.messages.append('LIGHT WHAT?')
        else:
            self.messages.append('YOU TRY TO LIGHT THE '+self.phrase)
            if self.phrase == 'CANDLE' and self.isCarrying(CANDLE) and not self.isCarrying(CANDLESTICK):
                self.messages.append('IT WILL BURN YOUR HAND!')
            elif self.phrase == 'CANDLE' and self.isCarrying(CANDLE) and not self.isCarrying(MATCHES):
                self.messages.append('YOU HAVE NOTHING TO LIGHT IT WITH')
            elif self.phrase == 'CANDLE' and self.isCarrying(CANDLE) and self.isCarrying(CANDLESTICK) and self.isCarrying(MATCHES):
                self.messages.append('IT CASTS A FLICKERING LIGHT')
                player.setFlag('CANDLELIT',True)
            else:
                self.messages.append('YOU CANNOT LIGHT THE '+self.phrase)
                
    def UNLIGHT(self):
        if self.isCarrying(CANDLE):
            if self.flags['CANDLELIT'] == True:
                self.setFlag('CANDLELIT',False)
                self.messages.append('YOU PUT THE CANDLE OUT. HISS!')
            else:
                self.messages.append('YOU\'RE TRYING TO PUT OUT AN EXTINGUISHED CANDLE? RIGHT-O!')
        else:
            self.messages.append('YOU HAVE NOTHING TO UNLIGHT')
            
    def SPRAY(self):
        if self.phrase == '':
            self.messages.append('SPRAY WHAT?')
        else:
            if self.phrase == 'BATS' and self.isVisible(BATS) and self.isHere(BATS):
                if self.isCarrying(AEROSOL):
                    self.messages.append('YOU SPRAY THE BATS WITH THE AEROSOL')
                    self.makeInvisible(BATS)
                else:
                    self.messages.append('YOU NEEED SOMETHING TO SPRAY THEM WITH')
            else:
                self.messages.append('IT IS NOT NICE TO GO AROUND SPRAYING THINGS')
                
    def USE(self):
        if self.phrase == '':
            self.messages.append('USE WHAT?')
        else:
            if self.phrase == 'VACUUM':
                if self.isCarrying(VACUUM):
                    if self.isCarrying(BATTERIES):
                        if self.flags['VACUUMON'] == True:
                            if self.isHere(GHOSTS) and self.isVisible(GHOSTS):
                                self.makeInvisible(GHOSTS)
                                self.messages.append('WHIZZZ! BYE BYE GHOSTIES!')
                            else:
                                self.messages.append('SO YOU HAVE HOOVERED THE HAUNTED HOUSE? FEEL LIKE YOU HAVE WASTED YOUR TIME?')
                        else:
                            self.flags['VACUUMON'] = True
                            self.messages.append('CLICK. BZZZ... THE VACUUM IS ON!')
                    else:
                        self.messages.append('BATTERIES NOT INCLUDED')
                else:
                    self.messages.append('YEAH YOU DON\'T HAVE ONE OF THEM')
            else:
                self.messages.append('YOU CANNOT USE THE '+self.phrase)

    def UNLOCK(self):
        if self.phrase == '':
            self.messages.append('UNLOCK WHAT?')
        else:
            if self.phrase == 'DESK' or self.phrase == 'DRAWER' and self.isHere(self.phrase):
                self.messages.append('THE DESK IS ALREADY UNLOCKED')
            else:
                if self.phrase == 'DOOR':
                    if self.isHere(DOOR):
                        if self.isCarrying(KEY):
                            if self.flags['DOORLOCKED'] == False:
                                self.messages.append('THE DOOR IS ALREADY UNLOCKED')
                            else:
                                self.messages.append('THE KEY TURNS AND YOU UNLOCK THE DOOR')
                                rooms[self.room].addExit('S',36)
                                self.setFlag('DOORLOCKED',False)
                        else:
                            self.messages.append('AND WHAT ARE YOU GOING TO UNLOCK THE DOOR WITH?')
                    else:
                        self.messages.append('WHAT DOOR WOULD THAT BE?')
                else:
                    self.messages.append('YOU CANNOT UNLOCK THE '+self.phrase)

Create objects

In [ ]:
def createObjects(rooms):
    #Gettable objects
    Object(PAINTING,'PAINTING',True,True).moveTo(rooms[46])
    Object(RING,'RING',True,False).moveTo(rooms[38])
    Object(MAGICSPELLS,'MAGIC SPELLS',True,True).moveTo(rooms[35])
    Object(GOBLET,'GOBLET',True,True).moveTo(rooms[50])
    Object(SCROLL,'SCROLL',True,True).moveTo(rooms[13])
    Object(COINS,'COINS',True,True).moveTo(rooms[18])
    Object(STATUE,'STATUE',True,True).moveTo(rooms[28])
    Object(CANDLESTICK,'CANDLESTICK',True,True).moveTo(rooms[42])
    Object(MATCHES,'MATCHES',True,True).moveTo(rooms[10])
    Object(VACUUM,'VACUUM',True,True).moveTo(rooms[25])
    Object(BATTERIES,'BATTERIES',True,True).moveTo(rooms[26])
    Object(SHOVEL,'SHOVEL',True,True).moveTo(rooms[12])
    Object(AXE,'AXE',True,True).moveTo(rooms[2])
    Object(ROPE,'ROPE',True,True).moveTo(rooms[7])
    Object(AEROSOL,'AEROSOL',True,True).moveTo(rooms[60])
    Object(CANDLE,'CANDLE',True,False).moveTo(rooms[43])
    Object(KEY,'KEY',True,False).moveTo(rooms[32])
    #None-gettable objects
    Object(DESK,'DESK',False,True).moveTo(rooms[43])
    Object(DRAWER,'DRAWER',False,False).moveTo(rooms[43])
    Object(HOLE,'HOLE',False,False).moveTo(rooms[43])
    Object(DOOR,'DOOR',False,False).moveTo(rooms[28])
    Object(COFFIN,'COFFIN',False,True).moveTo(rooms[38])
    Object(COAT,'COAT',False,True).moveTo(rooms[32])
    Object(RUBBISH,'RUBBISH',False,True).moveTo(rooms[3])
    Object(BOOKS,'BOOKS',False,False).moveTo(rooms[42])
    Object(WALL,'WALL',False,False).moveTo(rooms[43])
    Object(BATS,'BATS',False,False).moveTo(rooms[13])
    Object(GHOSTS,'GHOSTS',False,False).moveTo(rooms[52])
    return rooms

Create rooms

In [ ]:
def createRooms(rooms):
    rooms.append(Room(0,'DARK CORNER',{'S':8,'E':1},[]))
    rooms.append(Room(1,'OVERGROWN GARDEN',{'W':0,'E':2},[]))
    rooms.append(Room(2,'BY LARGE WOODPILE',{'W':1,'E':3},[]))
    rooms.append(Room(3,'YARD BY RUBBISH',{'S':11,'W':2,'E':4},[]))
    rooms.append(Room(4,'WEED PATCH',{'W':3,'E':5},[]))
    rooms.append(Room(5,'FOREST',{'W':4,'E':6},[]))
    rooms.append(Room(6,'THICK FOREST',{'S':14,'W':5,'E':7},[]))
    rooms.append(Room(7,'BLASTED TREE',{'W':6,'S':15},[]))
    rooms.append(Room(8,'CORNER OF HOUSE',{'N':0,'S':16},[]))
    rooms.append(Room(9,'ENTRANCE TO KITCHEN',{'S':17,'E':10},[]))
    rooms.append(Room(10,'KITCHEN & GRIMY COOKER',{'W':9,'E':11},[]))
    rooms.append(Room(11,'SCULLERY DOOR',{'N':3,'W':10},[]))
    rooms.append(Room(12,'ROOM WITH INCHES OF DUST',{'D':20,'E':13},[])) #D was originally S
    rooms.append(Room(13,'REAR TURRET ROOM',{'W':12},[]))
    rooms.append(Room(14,'CLEARING BY HOUSE',{'N':6,'E':15},[]))
    rooms.append(Room(15,'PATH',{'N':7,'S':23,'W':14},[]))
    rooms.append(Room(16,'SIDE OF HOUSE',{'N':8,'S':24},[]))
    rooms.append(Room(17,'BACK OF HALLWAY',{'N':9,'S':25},[]))
    rooms.append(Room(18,'DARK ALCOVE',{'S':26,'E':19},[]))
    rooms.append(Room(19,'SMALL DARK ROOM',{'W':18,'E':20},[]))
    rooms.append(Room(20,'BOTTOM OF SPIRAL STAIRCASE',{'W':19,'U':12},[])) #originally had N=U,D=W
    rooms.append(Room(21,'WIDE PASSAGE',{'S':29,'D':22},[])) #D originally E, but steps
    rooms.append(Room(22,'SLIPPERY STEPS',{'U':21,'D':30},[])) #originally U=W,D=S, but steps
    rooms.append(Room(23,'CLIFFTOP',{'N':15,'S':31},[]))
    rooms.append(Room(24,'NEAR CRUMBLING WALL',{'N':16},[]))
    rooms.append(Room(25,'GLOOMY PASSAGE',{'N':17,'S':33},[]))
    rooms.append(Room(26,'POOL OF LIGHT',{'N':18,'S':34,'E':27},[]))
    rooms.append(Room(27,'IMPRESSIVE VAULTED HALLWAY',{'W':26,'E':28},[]))
    rooms.append(Room(28,'HALL BY THICK WOODEN DOOR',{'W':27,'E':29},[]))
    rooms.append(Room(29,'TROPHY ROOM',{'N':21,'S':37,'W':28},[]))
    rooms.append(Room(30,'CELLAR WITH BARRED WINDOW',{'U':22,'S':38},[])) #U was N
    rooms.append(Room(31,'CLIFF PATH',{'N':23,'S':39},[]))
    rooms.append(Room(32,'CUPBOARD WITH HANGING COAT',{'S':40},[]))
    rooms.append(Room(33,'FRONT WALL',{'N':25,'S':41,'E':34},[]))
    rooms.append(Room(34,'SITTING ROOM',{'N':26,'S':42,'W':33},[]))
    rooms.append(Room(35,'SECRET ROOM',{'S':43},[]))
    rooms.append(Room(36,'STEEP MARBLE STAIRS',{'S':44,'D':28},[])) #D is one-way, was also N. U/S now just S
    rooms.append(Room(37,'DINING ROOM',{'N':29},[]))
    rooms.append(Room(38,'DEEP CELLAR WITH COFFIN',{'N':30},[]))
    rooms.append(Room(39,'CLIFF PATH',{'N':31,'S':47},[]))
    rooms.append(Room(40,'CLOSET',{'N':32,'E':41},[]))
    rooms.append(Room(41,'FRONT LOBBY',{'N':33,'W':40},[])) #Note incoming from 49 is one way
    rooms.append(Room(42,'LIBRARY OF EVIL BOOKS',{'N':34,'E':43},[]))
    rooms.append(Room(43,'STUDY WITH DESK & HOLE IN WALL',{'W':42},[]))
    rooms.append(Room(44,'WEIRD COBWEBBY ROOM',{'N':36,'S':52,'E':45},[]))
    rooms.append(Room(45,'VERY COLD CHAMBER',{'W':44,'E':46},[]))
    rooms.append(Room(46,'SPOOKY ROOM',{'W':46},[]))
    rooms.append(Room(47,'CLIFF PATH BY MARSH',{'N':39,'S':55},[]))
    rooms.append(Room(48,'RUBBLE-STREWN VERANDAH',{'S':56,'E':49},[]))
    rooms.append(Room(49,'FRONT PORCH',{'N':41,'S':57,'W':48},[]))
    rooms.append(Room(50,'FRONT TOWER',{'E':51},[]))
    rooms.append(Room(51,'SLOPING CORRIDOR',{'W':50,'E':52},[]))
    rooms.append(Room(52,'UPPER GALLERY',{'N':44,'W':51},[]))
    rooms.append(Room(53,'MARSH BY WALL',{'S':61},[]))
    rooms.append(Room(54,'MARSH',{'S':62,'W':53},[]))
    rooms.append(Room(55,'SOGGY PATH',{'N':47,'W':54},[]))
    rooms.append(Room(56,'BY TWISTED RAILING',{'N':48,'E':57},[]))
    rooms.append(Room(57,'PATH THROUGH IRON GATE',{'N':49,'W':56,'E':58},[]))
    rooms.append(Room(58,'BY RAILINGS',{'W':57,'E':59},[]))
    rooms.append(Room(59,'BENEATH THE FRONT TOWER',{'W':58,'E':60},[]))
    rooms.append(Room(60,'DEBRIS FROM CRUMBLING FACADE',{'W':59,'E':61},[]))
    rooms.append(Room(61,'LARGE FALLEN BRICKWORK',{'N':53,'W':60,'E':62},[]))
    rooms.append(Room(62,'ROTTING STONE ARCH',{'N':54,'W':61,'E':63},[]))
    rooms.append(Room(63,'CRUMBLING CLIFFTOP',{'W':62},[]))
    return rooms

Setup flags

In [ ]:
def setupFlags(player):
    player.setFlag('SAIDMAGICIN45',False)
    player.setFlag('ISUPTREE',False)
    player.setFlag('CANDLELIT',False)
    player.setFlag('VACUUMON',False)
    player.setFlag('DOORLOCKED',True)
    return player

Main program

In [ ]:
DEBUG = 1
from IPython.display import clear_output
from random import randint
import re

#Constants
MAGICWORD = 'XZANFAR'
PAINTING = 1
RING = 2
MAGICSPELLS = 3
GOBLET = 4
SCROLL = 5
COINS = 6
STATUE = 7
CANDLESTICK = 8
MATCHES = 9
VACUUM = 10
BATTERIES = 11
SHOVEL = 12
AXE = 13
ROPE = 14
AEROSOL = 15
CANDLE = 16
KEY = 17
DESK = 18
DRAWER = 19
HOLE = 20
DOOR = 21
COFFIN = 22
COAT = 23
RUBBISH = 24
BOOKS = 25
WALL = 26
BATS = 27
GHOSTS = 28

rooms = [] #Create a list of rooms
rooms = createRooms(rooms) #Populate the list of rooms
rooms = createObjects(rooms) #Create objects and put them in the rooms

#Create a player and put in room 0
player = Player(3)
player = setupFlags(player)
while not player.won:
    if DEBUG:
        print('\nCLS...\n')
    else:
        clear_output()
    player.showMessages()
    player.look()
    player.getInstruction()